# Big Bird Classifier for all

In [1]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install torch --quiet
!pip install emoji --quiet
!pip install sentencepiece --quiet
!pip install --upgrade accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [4]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import warnings
import os
import re
import ast
import nltk
import emoji
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import BigBirdForTokenClassification, BigBirdTokenizerFast
from torch import cuda
from google.colab import drive
from sklearn.utils import shuffle
import torch

from nltk.corpus import stopwords

warnings.filterwarnings("ignore")

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data = pd.read_csv('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/raw/combined_balanced.csv')

In [8]:
data

,subreddit,post,label
0,control,What do you do when most of a class passes and...,0
1,control,"Picking a chapter book for ""cool down time"" I'...",0
2,control,How can I prevent alumni from visiting me? As ...,0
3,control,Why I feel like a dumb student In elementary S...,0
4,control,Anonymous questionnaire on Social Anxiety Hell...,0
...,...,...,...
143995,bipolarreddit,Business travel w/ bipolar? Hi Friends -\n\nI'...,3
143996,bipolarreddit,Finding a job when depressed is impossible. Fi...,3
143997,bipolarreddit,Mania?? I was diagnosed with bipolar over 4 ye...,3
143998,bipolarreddit,Are any of you successfully managing your cond...,3


In [9]:
data['subreddit'].unique()

array(['control', 'depression', 'anxiety', 'adhd', 'bpd', 'EDAnonymous',
       'schizophrenia', 'ptsd', 'bipolarreddit'], dtype=object)

In [10]:
data['subreddit'] = data['subreddit'].str.replace('EDAnonymous','eda')
data['subreddit'] = data['subreddit'].str.replace('bipolarreddit','bipolar')

In [11]:
data['subreddit'].unique()

array(['control', 'depression', 'anxiety', 'adhd', 'bpd', 'eda',
       'schizophrenia', 'ptsd', 'bipolar'], dtype=object)

In [12]:
train_text_df  = data.rename(columns={'subreddit': 'entities', 'post': 'text', 'label': 'labels'})

In [13]:
train_text_df ["id"] = train_text_df .index + 1

In [14]:
train_text_df ['entities'] = train_text_df ['entities'].str.strip('()').str.split(',')

In [15]:
train_text_df = shuffle(train_text_df)

In [16]:
train_text_df.drop(['labels'], axis=1, inplace=True)

In [17]:
train_text_df = train_text_df.reset_index(drop=True)

In [18]:
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
train_text_df['text'] = train_text_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train_text_df['text'] = train_text_df['text'].str.lower()
train_text_df['text'] = train_text_df['text'].apply(lambda s: emoji.replace_emoji(s, ''))
train_text_df['text'] = train_text_df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
train_text_df['text'] = train_text_df['text'].str.replace('[^a-zA-Z\ ]', '')

In [20]:
train_text_df 

,entities,text,id
0,[adhd],got into medical school struggling college alm...,61285
1,[depression],in hours school starts then time see plan tod...,23487
2,[bipolar],idk i dont want therapy meds anymore i dont wa...,138688
3,[eda],green tea amazing thats thats fucking post,94618
4,[bipolar],terrified doctor appointment tomorrow i going ...,132357
...,...,...,...
143995,[depression],i wish friends knew i hate fact emotional turm...,20468
143996,[depression],broken ive deal lot loss last years mother di...,31598
143997,[schizophrenia],misdiagnosed schizophrenic well guys aripipraz...,110260
143998,[bipolar],i know i getting depressed when i want exercis...,134169


In [21]:
train_text_df['entities'] = train_text_df['entities'].apply(lambda x:ast.literal_eval(str(x)))

print(train_text_df['entities'].values[0])

['adhd']


In [22]:
train_text_df = train_text_df[:]

In [23]:
config = {'model_name': 'google/bigbird-roberta-base',
         'max_length': 150,
         'train_batch_size':4,
         'valid_batch_size':4,
         'epochs':5,
         'learning_rate':5e-05,
         'max_grad_norm':10,
          'warmup':0.1,
          "grad_acc":8,
          "model_save_path":"big-bird",
         'device': 'cuda' if cuda.is_available() else 'cpu'}

output_labels = ['control', 'depression', 'anxiety', 'adhd', 'bpd', 'eda',
       'schizophrenia', 'ptsd', 'bipolar']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [24]:
train_text_df['labels'] = train_text_df['entities'].apply(lambda x: [labels_to_ids[i] for i in x])

In [25]:
train_text_df.head()

,entities,text,id,labels
0,[adhd],got into medical school struggling college alm...,61285,[3]
1,[depression],in hours school starts then time see plan tod...,23487,[1]
2,[bipolar],idk i dont want therapy meds anymore i dont wa...,138688,[8]
3,[eda],green tea amazing thats thats fucking post,94618,[5]
4,[bipolar],terrified doctor appointment tomorrow i going ...,132357,[8]


In [26]:
tokenizer = BigBirdTokenizerFast.from_pretrained(config['model_name'])
model = BigBirdForTokenClassification.from_pretrained(config['model_name'],
                                                     num_labels=len(output_labels))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

In [27]:
converted = tokenizer(train_text_df.loc[0].values[1].split(),
                      is_split_into_words=True, return_offsets_mapping=True)

In [28]:
ix = 0
for i,j in zip(tokenizer.convert_ids_to_tokens(converted['input_ids']), converted['offset_mapping']):
    print(i, j)
    ix += 1
    if ix == 15:
        break

[CLS] (0, 0)
▁got (0, 3)
▁into (0, 4)
▁medical (0, 7)
▁school (0, 6)
▁struggling (0, 10)
▁college (0, 7)
▁almost (0, 6)
▁sure (0, 4)
▁i (0, 1)
▁type (0, 4)
▁ad (0, 2)
hd (2, 4)
▁getting (0, 7)
▁diagnosed (0, 9)


In [29]:
def tokenizer_data(example):
    encoding = tokenizer(example['text'].split(),
                         is_split_into_words=True,
                         truncation=True,
                         padding='max_length', 
                         return_offsets_mapping=True,
                         max_length=config['max_length'])
    i = 0
    labels = example['labels']
    encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
    for idx, mapping in enumerate(encoding["offset_mapping"]):
        if mapping[0] == 0 and mapping[1] != 0:
            try:
                encoded_labels[idx] = labels[i]
            except:
                pass
            i += 1
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    item['labels'] = torch.as_tensor(encoded_labels)
    return item

In [30]:
dataset = datasets.Dataset.from_pandas(train_text_df)

In [31]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 129600
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 14400
    })
})

In [32]:
text = dataset['train'][1]

In [33]:
converted = tokenizer_data(text)

converted

{'input_ids': tensor([   65, 14495,   357, 32930,  4699,  2098,  1413,  7461,   866,  2248,
          1413,  2801,   752,  4097,  1335,  6992,   420,  6677,  2217,   752,
          4089,  1625,  2614,  4709,  4546,  1413,  1344,  1413,  4486,   385,
          1413,  6328,  2098,  1413,  4702,  4094,  1211,   891,  1211,  2248,
          2174,   578,  1344,  1074,  2323,  8817,  1413,  3037, 15448,  1846,
          5299,   624,  1413,  1751,  2443,  1096,  1309,  1355,  8027,   441,
           889,  1413,  1355, 15035, 29316,  1966,  4945,  6657,   790,   752,
         18608,  2098,   578,  1413,  6032, 24655, 19622,   644,  1819,   913,
          4574,  4150,  1165, 31271,  6150,  2020,  9852,  2248,  4048,   888,
         18895, 26882,  1506,   442, 38054,  8963,  1413,  5126,  2364,  2380,
          2034,  2185,  1413,  6260,  1466,  4886,   529, 14170, 36669,  4572,
          1305,  1413,  9776, 38806,  7561,  5646,  2506,   646,   736,  2149,
          7826,  9341,  2870,  3324,  7

In [34]:
converted

i=0
for token, label in zip(tokenizer.convert_ids_to_tokens(converted["input_ids"]), 
                        converted["labels"]):
    print(token, label, converted['offset_mapping'][i])
    i+=1
    if i == 15:
        break

[CLS] tensor(-100) tensor([0, 0])
▁lacking tensor(8) tensor([0, 7])
▁t tensor(-100) tensor([0, 1])
owell tensor(-100) tensor([1, 6])
do tensor(-100) tensor([6, 8])
▁anything tensor(-100) tensor([0, 8])
▁i tensor(-100) tensor([0, 1])
▁literally tensor(-100) tensor([0, 9])
▁want tensor(-100) tensor([0, 4])
▁nothing tensor(-100) tensor([0, 7])
▁i tensor(-100) tensor([0, 1])
▁force tensor(-100) tensor([0, 5])
▁get tensor(-100) tensor([0, 3])
▁bed tensor(-100) tensor([0, 3])
▁put tensor(-100) tensor([0, 3])


In [35]:
dataset = dataset.map(tokenizer_data)

dataset

Map:   0%|          | 0/129600 [00:00<?, ? examples/s]

Map:   0%|          | 0/14400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 129600
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 14400
    })
})

In [36]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',
                                         'labels'])

dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 129600
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 14400
    })
})

In [37]:
for a in dataset['train']:
    if a['input_ids'].shape[0] != a['attention_mask'].shape[0]:
        print(a)
        break

In [38]:
trainer_args = TrainingArguments(report_to='none',
                                output_dir="/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/classificator/BigBird/All_V2",
                                num_train_epochs=config['epochs'],
                                evaluation_strategy ='epoch',
                                per_device_train_batch_size=config['train_batch_size'],
                                per_device_eval_batch_size=config['valid_batch_size'],
                                fp16=True,
                                save_strategy = "epoch",
                                warmup_ratio= config['warmup'],
                                gradient_accumulation_steps=config['grad_acc'],
                                logging_strategy="epoch",
                                save_total_limit=1
                                )

trainer = Trainer(model=model,
                  args=trainer_args, 
                  train_dataset = dataset['train'],
                  eval_dataset=dataset['test'],
                  tokenizer=tokenizer)

In [39]:
trainer.train()

You're using a BigBirdTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 150 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss
1,0.715700,0.480294
2,0.412600,0.400565
3,0.258700,0.368844
4,0.154300,0.380496
5,0.087300,0.423646


TrainOutput(global_step=20250, training_loss=0.32570299087336035, metrics={'train_runtime': 12434.18, 'train_samples_per_second': 52.114, 'train_steps_per_second': 1.629, 'total_flos': 4.99531287816e+16, 'train_loss': 0.32570299087336035, 'epoch': 5.0})

In [40]:
device = config['device']

In [41]:
trainer.model.eval()
def inference(sentence):
    inputs = tokenizer(sentence.split(),
                        is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=200,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = trainer.model(input_ids=ids, attention_mask=mask, return_dict=False)
#     print(outputs)
    logits = outputs[0]
    
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    print(logits.shape, active_logits.shape, flattened_predictions.shape)
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [42]:
#Anxiety
text_1= 'Ive never taken it to a doctor, but Ive heard that it could help for this problem, but I dont want to take it to the point where I am constantly afraid of dying.'
pred_1, _ = inference(text_1)
print(pred_1[0])

torch.Size([1, 200, 9]) torch.Size([200, 9]) torch.Size([200])
anxiety


In [46]:
#ADHD
text_2= 'I am not sure if this means that I am not sick or if I am just sad and have no sense of direction.'
pred_2, _ = inference(text_2)
print(pred_2[0])

torch.Size([1, 200, 9]) torch.Size([200, 9]) torch.Size([200])
depression
